In [15]:
import rebound
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import cm
plt.style.use('custom_paper')

In [16]:
def about180(aray):
    out = []
    for val in aray:
        while val < 0:
            val += 2*np.pi
        while val > 2*np.pi:
            val -= 2*np.pi
        out.append(val)
    return ((np.asarray(out)-np.pi)*180/np.pi)

In [17]:
sim = rebound.Simulation()
sim.units = ('AU', 'Yr', 'Msun')
sim.add(m=1) 
sim.add(m=1e-4,a=1,e=0.01,pomega=-np.pi,f=3*np.pi/2) #circular jupiter

j,k = 3,1
a0 = (j/(j-k))**(2/3)
npar = 50
delta= 4e-2

agrid = np.linspace(a0-delta,a0+delta,npar)
egrid = np.linspace(0.01,0.20,npar)

for a in agrid:
    for e in egrid:
        sim.add(m=0,a=a,e=e,omega=0)

In [18]:
sim.integrator = "whfast"
sim.dt = 0.05 * sim.particles[1].P  # 5% of planet's period
Nout = 10000            # number of points to display
tmax = np.pi*100         # integrate for 100pi years, make this irrational so the animation is smooth

In [19]:
Ntest = npar**2
As = np.zeros((Ntest+1,Nout))
Periods = np.zeros((Ntest+1,Nout))
Es = np.zeros((Ntest+1,Nout))
Ls = np.zeros((Ntest+1,Nout))
Ps = np.zeros((Ntest+1,Nout))

times = np.linspace(0.,tmax,Nout)
ps = sim.particles

for i,time in enumerate(times):
    sim.integrate(time)
    os = sim.calculate_orbits()
    for j in range(Ntest+1): #first particle is the planet
        As[j][i] = os[j].a 
        Periods[j][i] = os[j].P 
        Es[j][i] = os[j].e
        Ls[j][i] = os[j].l
        Ps[j][i] = os[j].pomega

In [20]:
theta_e2 = 3.*Ls[1:] - 2.*Ls[0] - Ps[1:]
theta_e2 = np.array([about180(te2) for te2 in theta_e2])

amplitudes= np.max(theta_e2,axis=1)
colors = cm.viridis(amplitudes/180)
colors[amplitudes > 179] = [1,0,0,1]

In [35]:
def make_figures(i):
    fig,ax = plt.subplots(figsize=(15,7))
    ax.scatter(Periods[1:][:,i],Es[1:][:,i],c=colors)
    sc = ax.scatter(Periods[1:][:,i],Es[1:][:,i],c=amplitudes,s=0)
    plt.colorbar(sc,label=r'Libration Amplitude [$^\circ$]')
    ax.set_xlabel(r"$P_2/P_1$")
    ax.set_ylabel("Eccentricty")
    ax.set_xlim(np.min(Periods[1:][:,0]),np.max(Periods[1:][:,0]))
    ax.set_ylim(np.min(Es[1:][:,0]),np.max(Es[1:][:,0]))
    ax.set_title("t = "+str(np.round(times[i],1))+" [yrs]")
    return fig

In [36]:
from animate import *

In [37]:
parameter_grid = np.arange(0,len(times),33)

In [38]:
savefigures(make_figures,parameter_grid,outdir='./tornado_plot/',dpival=100)

saving figures


100%|█████████████████████████████████████████| 304/304 [00:56<00:00,  5.39it/s]


In [40]:
render('./tornado_plot/','tornado_plot',output_type='gif',cleanup_type='rm')

Framerate is None fps
Total Runtime is None s


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab